In [16]:
# -------------------------------------------------------------------------------
# Read POS data
# Read COnfiguration data
# -------------------------------------------------------------------------------
# Compute Frequency and Monetary values for ecah Customer
# Compute Normalised Frequency and Monetary values for each Customer
# Read Customer Segments and their corresponding Frequency, Monetary ranges from configuration file
# Map the Customer Segments to the Customers based on these F and M ranges
# For ease of use, all values of F and M ranges can be specified as a Normalised value between 1 and 100
# Auto Segmentation of Customers
# Auto segmentation applied on Normalised F and M values of Customers to group them into Customer segments
# -------------------------------------------------------------------------------
# Import libraries
import csv
import pandas as pd
import random as r
import sklearn
from sklearn.cluster import KMeans


In [20]:
#---------------
# Function to Read Customer segments data from Configuration file
#  CustomerSegment|SegmentAutoRank|RFrom|RTo|FFrom|FTo|MFrom|MTo
#  Platinum|1|40|50|40|50|40|50
#  Loyal Gold|2|20|50|30|50|30|50

def read_custsegments_config(in_file_obj):
    df = pd.read_csv(in_file_obj, header='infer', delimiter='|', skip_blank_lines=True, parse_dates=True, dtype=str)
    df = df.sort_values(by=['SegmentAutoRank'])
    return(df)

#---------------
# Compute Customer statistics like Frequency, Monetary value or Total spend by each Customer
#    for further analysis
def customer_stats(df_pos):
	df = df_pos
	#Non negative filter
	df_nn = df[df["UnitPrice"]>0]
	# Group
	v_lineval=df_nn["Quantity"].astype(float) * df_nn["UnitPrice"].astype(float)
	df_nn=df_nn.assign(LineValue=v_lineval)
	#Non negative filter
	df_nn = df_nn[df_nn["LineValue"]>0]
	# Group the data frame by month and item and extract a number of stats from each group
	grouped_df = df_nn.groupby('CustomerID', as_index=False).agg({'LineValue': [min, max, sum],      # find the min, max, and sum of the duration column
					   'InvoiceNo': ["count"], # find the number of network type entries
					   'InvoiceDate': [min, max, 'first', 'nunique']})    # get the min, first, and number of unique dates per group
	grouped_df.columns = ["".join(x) for x in grouped_df.columns.ravel()]
	grouped_df.columns
	grouped_df.head()
	return(grouped_df)

#---------------
# Normalize functions
# Used for Normalising the absolute RFM values into a range of 1 to 100
# This Normalisation range between 1 to 100 will help visualise the F,M 
#                                           values intuitively by users

def fxnorm(x, mina, maxa):
    x_norm = (((x-float(mina))/(float(maxa)-float(mina)))*100)
    return x_norm

def flnorm(l, minx, maxx):
    l_norm = [fxnorm(i,minx, maxx) for i in l]
    return l_norm

# flnorm(x, min(x), max(x))
#---------------
# Compute Customer Normals
def customer_normals(df_custstats):
    # Normalize Customer stat 1
    custvals=df_custstats.LineValuesum
    v_ValsNorm=flnorm(custvals, min(custvals), max(custvals))
    df_custstats=df_custstats.assign(LineValuesumNorm=v_ValsNorm)
    # Normalize Customer stat 2
    custvals=df_custstats.InvoiceNocount
    v_ValsNorm=flnorm(custvals, min(custvals), max(custvals))
    df_custstats=df_custstats.assign(InvoiceNocountNorm=v_ValsNorm)
    # Check Dataset
    df_custstats.columns
    df_custstats.head()
    #df_analyze = pd.DataFrame({'CustomerID':grouped_df['CustomerID'].values,
    #                           'LineValuesumNorm':grouped_df['LineValuesumNorm'].values,
    #                           'InvoiceNocountNorm':grouped_df['InvoiceNocountNorm'].values})
    df_analyze = pd.DataFrame({'LineValuesumNorm':df_custstats['LineValuesumNorm'].values,
                               'InvoiceNocountNorm':df_custstats['InvoiceNocountNorm'].values})
    return(df_analyze)


#---------------
# To conduct experiments with Automatic Segmentation of Customers based on their 
#  Frequency and Monetary values
# Automatic Customer segmentation and Ranking using k-means algorithm
#  Segment count is picked up from the CustomerSegment configuration file
#  But the Normalised F and M value ranges defined in the configuration file is ignored in Auto Ranking
    
def auto_cust_ranks(df_analyze, df_custstats, in_clustcount=2):
    from sklearn.cluster import KMeans
    # Convert DataFrame to matrix
    mat = df_analyze.as_matrix()
    # Using sklearn
    km = sklearn.cluster.KMeans(n_clusters=in_clustcount)
    km.fit(mat)
    # Get cluster assignment labels
    labels = km.labels_
    # Format results as a DataFrame
    results = pd.DataFrame([df_analyze.index,labels]).T
    results  
    results[0]  
    print(df_analyze.head())  
    # Get the Customer groups  
    # Join the Groups  
    df_CustGroups = pd.DataFrame({'CustomerGroup':results[1], 'CustomerID':df_custstats['CustomerID'].values,
                                  'F':df_custstats['InvoiceNocount'].values, 'M':df_custstats['LineValuesum'].values,
                                  'FNorm':df_custanalyze['InvoiceNocountNorm'].values, 'MNorm':df_custanalyze['LineValuesumNorm'].values})  
    df_CustGroups = df_CustGroups.sort_values(by=['CustomerGroup', 'MNorm', 'FNorm'])  
    return(df_CustGroups)  

# Compute Customer Segment statistics
def segment_stats(df_custRanked):
    df = df_custRanked
    # Group the data frame by month and item and extract a number of stats from each group
    # find the mean
    grouped_df = df.groupby('CustomerGroup', as_index=False).agg({'F': ['mean'], 'FNorm': ['mean'],      
                           'M': ['mean'], 'MNorm': ['mean'] })
    grouped_df.columns = ["".join(x) for x in grouped_df.columns.ravel()]
    grouped_df.columns
    print(grouped_df.head())
    # grouped_df.sort_values(by=['CustomerGroup', 'MNormmean', 'FNormmean'])
    grouped_df = grouped_df.sort_values(by=['MNormmean', 'FNormmean'], ascending=False)
    print(grouped_df)
    return(grouped_df)

# Assign Auto segment names
# df_segmentstats
# df_CustGroups = pd.DataFrame({'CustomerGroup':df_custRanked['CustomerGroup'].values, 'CustomerID':df_custstats['CustomerID'].values})
# df_CustGroups.head()
def auto_cust_segments(df_custRanked, df_segmentstats):
    # del df_custRanked['CustomerSegment']
    df_custRanked.head()
    df_custRanked = df_custRanked.assign(CustomerSegment=("Unassigned"))
    df_custRanked.head()
    # --- Assign in a loop
    for i, row in df_segmentstats.iterrows():
        print (row.CustomerSegment, row.CustomerGroup)
        df_custRanked.loc[(df_custRanked["CustomerGroup"]==row.CustomerGroup), 'CustomerSegment']=row.CustomerSegment
    df_custRanked.head()
    return(df_custRanked)

# Manual mapping ofCustomers to Customer segments
#   Normalised F and M values defined in CustomerSegment configuration file used
#   to map Customers based on their Normalised F and M scores
def manual_cust_segments(df_custRanked, df_custseg):
    df_custRanked.dtypes
    # Add a column CustomerSegment to Ranked Customer dataset
    if 'CustomerSegment' in df_custRanked.columns:
        del df_custRanked['CustomerSegment']
    df_custRanked.head()
    df_custRanked = df_custRanked.assign(CustomerSegment=("Unassigned"))
    df_custRanked.head()
    # Filter Customer Segments for manual matching
    for i, row in df_custseg.iterrows():
        # print (row.CustomerSegment, row.SegmentAutoRank, row.RFrom, row.RTo, row.MFrom, row.MTo)
        # print(df_custRanked[(df_custRanked["FNorm"]>=float(row.FFrom)) & (df_custRanked["FNorm"]<float(row.FTo)) &
        #                    (df_custRanked["MNorm"]>=float(row.MFrom)) & (df_custRanked["MNorm"]<float(row.MTo))])
        # df_custRanked[(df_custRanked["FNorm"]>=float(row.FFrom)) & (df_custRanked["FNorm"]<float(row.FTo)) &
        #                    (df_custRanked["MNorm"]>=float(row.MFrom)) & (df_custRanked["MNorm"]<float(row.MTo))].CustomerSegment
        df_custRanked.loc[(df_custRanked["FNorm"]>=float(row.FFrom)) & (df_custRanked["FNorm"]<float(row.FTo)) &
                            (df_custRanked["MNorm"]>=float(row.MFrom)) & (df_custRanked["MNorm"]<float(row.MTo)), 'CustomerSegment']=row.CustomerSegment
    return(df_custRanked)

# -------------------------------------------------------------------


In [36]:
# Wrapper function to run Customer Segmentation
def segment_customers(df_pos, df_custsegconfig, in_segment_mode='AUTO'):
# Compute Customer metrics
    df_custstats = customer_stats(df_pos)
    df_custstats.head()
    # NormalizeCustomer metrics
    df_custanalyze = customer_normals(df_custstats)
    df_custanalyze.head()
    # Automatic Segmentation of Customers
    v_clustcount = len(df_custsegconfig)
    print("Number of Clusters for Auto Segmentation : ", v_clustcount)
    df_custRanked = auto_cust_ranks(df_custanalyze, df_custstats, v_clustcount)
    df_custRanked.head()
    # Execute flow for AUTO segmentation or MANUAL segmentation    
    if (in_segment_mode == 'AUTO'):    
        # Option 1: AUTO segmentation of Customers
        # Customer segment wise stats
        df_segmentstats = segment_stats(df_custRanked)
        df_segmentstats.head()
        # Map Auto SegmentGroup with identifyable names from Configuration file
        df_segmentstats = df_segmentstats.assign(CustomerSegment=df_custsegconfig['CustomerSegment'].values)
        df_segmentstats.head()
        df_segmentstats.dtypes
        # Assign Auto Cutomersegment statsto Customer groups
        df_SegmentedCustomers = auto_cust_segments(df_custRanked, df_segmentstats)
        df_SegmentedCustomers.head()
    else:
        # Option 2: MANUAL segmentation of Customers
        df_SegmentedCustomers = manual_cust_segments(df_custRanked, df_custsegconfig)
        df_SegmentedCustomers.head()
    return(df_SegmentedCustomers)



In [2]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ec7832a631cf488c8a552db7884026b6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9lpMGaOqG9N_xYlDNyNKYxVNjfKoxX5iTFFW-5bZ1NgA',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ec7832a631cf488c8a552db7884026b6.get_object(Bucket='campaignmanagementded83c5c44d440ebba56858d64bcdce0',Key='Online Retail Sample.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_pos = pd.read_csv(body)
df_pos.head()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26:00,3.39,17850.0,United Kingdom


In [31]:
# Read the configuration file
# ---------------------------
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_ec7832a631cf488c8a552db7884026b6.get_object(Bucket='campaignmanagementded83c5c44d440ebba56858d64bcdce0', Key='PromoConfig.txt')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 

df_custsegconfig = pd.read_csv(streaming_body_1, header='infer', delimiter='|', skip_blank_lines=True, parse_dates=True, dtype=str)
df_custsegconfig = df_custsegconfig.sort_values(by=['SegmentAutoRank'])
df_custsegconfig.head()

,CustomerSegment,SegmentAutoRank,RFrom,RTo,FFrom,FTo,MFrom,MTo
0,Platinum,1,40,50,40,50,40,50
1,Loyal Gold,2,20,50,30,50,30,50
2,Up Sell,3,40,50,40,50,10,30
3,Cross Sell,4,30,40,0,10,0,10
4,Attend Now,5,20,30,20,30,20,30


In [34]:
# AUTO Segmentation of Customers
df_custsegments_auto = segment_customers(df_pos, df_custsegconfig, in_segment_mode='AUTO')
df_custsegments_auto.head()

Number of Clusters for Auto Segmentation :  7
   InvoiceNocountNorm  LineValuesumNorm
0           15.476190          5.369130
1           85.714286         29.090105
2           22.619048         12.931350
3           16.666667          3.898508
4            0.000000          0.000000
   CustomerGroup  FNormmean      Fmean        Mmean   MNormmean
0              0  17.784993  15.939394   414.502727    6.224007
1              1  94.940476  80.750000  1052.560000   15.920617
2              2   4.497354   4.777778   211.468056    3.138472
3              3  38.095238  33.000000  6585.160000  100.000000
4              4  70.634921  60.333333   387.246667    5.809794
   CustomerGroup  FNormmean      Fmean        Mmean   MNormmean
3              3  38.095238  33.000000  6585.160000  100.000000
6              6  17.063492  15.333333  2667.533333   40.463501
1              1  94.940476  80.750000  1052.560000   15.920617
0              0  17.784993  15.939394   414.502727    6.224007
4         

,CustomerGroup,CustomerID,F,FNorm,M,MNorm,CustomerSegment
27,0,14078.0,13,14.285714,136.24,1.995225,Cross Sell
49,0,15605.0,12,13.095238,194.85,2.885926,Cross Sell
84,0,17905.0,23,26.190476,201.75,2.990786,Cross Sell
7,0,12868.0,12,13.095238,203.30,3.014341,Cross Sell
11,0,13065.0,14,15.476190,205.86,3.053246,Cross Sell


In [38]:
# df_custsegconfig.head()
# Manual Segmentation of Customers based on Customer Segment Configuration
df_custsegments_manual = segment_customers(df_pos, df_custsegconfig, in_segment_mode='MANUAL')


Number of Clusters for Auto Segmentation :  7
   InvoiceNocountNorm  LineValuesumNorm
0           15.476190          5.369130
1           85.714286         29.090105
2           22.619048         12.931350
3           16.666667          3.898508
4            0.000000          0.000000
